In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
import random
import time as t
import os
from subprocess import Popen

In [183]:
ControlSparkCluster('stop')

cluster stopped


In [172]:
def ControlSparkCluster(action):
    if action == 'start':
        Popen('./start_cluster.sh',shell=True)
        t.sleep(10)
        print 'cluster started'
    elif action == 'stop':
        Popen('./stop_cluster.sh',shell=True)
        sc.stop()
        t.sleep(3)
        print 'cluster stopped'
    else:
        raise Exception('Unsupported command!')

In [174]:
ControlSparkCluster('start')

cluster started


In [175]:
conf = SparkConf().setMaster("spark://cathome:7077").setAppName("bosh")\
.setAll((['spark.executor.cores','4']
        ,['spark.driver.cores','4']
        ,['spark.driver.memory','1g']
        ,['spark.executor.memory','1g']
        ,['spark.python.worker.memory','2g']
        ,['spark.default.parallelism','4']
       ))

In [176]:
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

In [177]:
dir_name = '../../../diploma_data'

In [178]:
tr_file_name = 'train_numeric.csv'
ts_file_name = 'test_numeric.csv'

In [179]:
if tr_file_name in os.listdir(dir_name):# and ts_file_name in os.listdir(dir_name):
    tr_data_file = '{0}/{1}'.format(dir_name,tr_file_name)
    #ts_data_file = '{0}/{1}'.format(dir_name,ts_file_name)
else:
    raise Exception('No such file!')

In [180]:
%%time
df = sqlContext.read.csv(tr_data_file, sep=',', header=True, inferSchema=True)

CPU times: user 8 ms, sys: 8 ms, total: 16 ms
Wall time: 1min 5s


In [182]:
l = df.select(df.Id,df.Response).collect() 

In [181]:
df2 = df
df2.limit(10).toPandas()

,Id,L0_S0_F0,L0_S0_F2,L0_S0_F4,L0_S0_F6,L0_S0_F8,L0_S0_F10,L0_S0_F12,L0_S0_F14,L0_S0_F16,...,L3_S50_F4245,L3_S50_F4247,L3_S50_F4249,L3_S50_F4251,L3_S50_F4253,L3_S51_F4256,L3_S51_F4258,L3_S51_F4260,L3_S51_F4262,Response
0,4,0.030,-0.034,-0.197,-0.179,0.118,0.116,-0.015,-0.032,0.020,...,None,None,None,None,None,None,None,None,None,0
1,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0
2,7,0.088,0.086,0.003,-0.052,0.161,0.025,-0.015,-0.072,-0.225,...,None,None,None,None,None,None,None,None,None,0
3,9,-0.036,-0.064,0.294,0.330,0.074,0.161,0.022,0.128,-0.026,...,None,None,None,None,None,None,None,None,None,0
4,11,-0.055,-0.086,0.294,0.330,0.118,0.025,0.030,0.168,-0.169,...,None,None,None,None,None,None,None,None,None,0
5,13,0.003,0.019,0.294,0.312,0.031,0.161,0.022,0.088,-0.005,...,None,None,None,None,None,None,None,None,None,0
6,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0
7,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0
8,18,-0.016,-0.041,-0.179,-0.179,-0.056,0.161,-0.007,-0.032,-0.082,...,None,None,None,None,None,None,None,None,None,0
9,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,None,None,None,None,None,None,None,None,None,0


In [156]:
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [157]:
features = df2.columns[1:-1]

In [158]:
vecAssembler = VectorAssembler(inputCols=features, outputCol="features")

In [159]:
df2  = df2.na.fill(99999999)

In [160]:
%%time
ass = vecAssembler.transform(df2)

CPU times: user 96 ms, sys: 12 ms, total: 108 ms
Wall time: 1.14 s


In [161]:
assd = ass.select(['Id','Response','features']).drop()

In [162]:
assd.limit(5).toPandas()
assd.printSchema()

,Id,Response,features
0,4,0,"[0.03, -0.034, -0.197, -0.179, 0.118, 0.116, -..."
1,6,0,"[99999999.0, 99999999.0, 99999999.0, 99999999...."
2,7,0,"[0.088, 0.086, 0.003, -0.052, 0.161, 0.025, -0..."
3,9,0,"[-0.036, -0.064, 0.294, 0.33, 0.074, 0.161, 0...."
4,11,0,"[-0.055, -0.086, 0.294, 0.33, 0.118, 0.025, 0...."


root
 |-- Id: integer (nullable = true)
 |-- Response: integer (nullable = true)
 |-- features: vector (nullable = true)



In [163]:
tr_dt,tst_dt = assd.randomSplit([0.7,0.3])

In [164]:
gbt = GBTClassifier(maxIter=5, maxDepth=2, labelCol="Response", seed=42)

In [165]:
%%time
model = gbt.fit(tr_dt)

CPU times: user 112 ms, sys: 36 ms, total: 148 ms
Wall time: 16min 26s


In [166]:
model.featureImportances

SparseVector(968, {341: 0.3003, 356: 0.087, 394: 0.0424, 765: 0.1811, 848: 0.215, 855: 0.041, 883: 0.1333})

In [167]:
pr = model.transform(tst_dt)

In [169]:
pr.show()

+---+--------+--------------------+----------+
| Id|Response|            features|prediction|
+---+--------+--------------------+----------+
|  4|       0|[0.03,-0.034,-0.1...|       0.0|
|  6|       0|[9.9999999E7,9.99...|       0.0|
| 14|       0|[9.9999999E7,9.99...|       0.0|
| 47|       0|[9.9999999E7,9.99...|       0.0|
| 52|       0|[9.9999999E7,9.99...|       0.0|
| 55|       0|[9.9999999E7,9.99...|       0.0|
| 71|       0|[-0.167,-0.168,0....|       0.0|
| 78|       0|[0.101,0.108,0.00...|       0.0|
| 79|       0|[9.9999999E7,9.99...|       0.0|
| 81|       0|[0.036,0.078,-0.1...|       0.0|
| 91|       0|[0.036,0.101,-0.1...|       0.0|
| 92|       0|[0.108,0.093,-0.0...|       0.0|
| 95|       0|[0.088,0.071,-0.0...|       0.0|
|103|       0|[-0.055,-0.041,-0...|       0.0|
|115|       0|[0.075,0.056,-0.1...|       0.0|
|123|       0|[-0.023,-0.034,-0...|       0.0|
|126|       0|[0.095,0.056,-0.1...|       0.0|
|131|       0|[-0.003,0.041,0.3...|       0.0|
|146|       0

In [168]:
label_pred_list = pr.select(pr['Response'],pr['prediction']).collect()
matthews_corr_coef(label_pred_list)

True Positives: 6
True Negatives: 352534
False Positives: 0
False Negatives: 2040
Matthews correlation coefficient:  0.0539970298159


In [121]:
def matthews_corr_coef(label_pred_list): 
    
    tp = np.sum(1 for r in label_pred_list if r[0] == 1 and r[1] == 1)
    tn = np.sum(1 for r in label_pred_list if r[0] == 0 and r[1] == 0)
    fp = np.sum(1 for r in label_pred_list if r[0] == 0 and r[1] == 1)
    fn = np.sum(1 for r in label_pred_list if r[0] == 1 and r[1] == 0)
    print "True Positives:", tp
    print "True Negatives:", tn
    print "False Positives:", fp
    print "False Negatives:", fn
    #print "Total", df.count()
    try:
        MCC = ((tp * tn) - (fp * fn)) / np.sqrt((tp + fp)*(tp + fn)*(tn + fp)*(tn + fn))
    except ZeroDivisionError:
        MCC = -1
    print "Matthews correlation coefficient: ", MCC